In [1]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path
SEED_XML_DIR = '/data/darknet/python/xml/'
SEED_IMG_DIR = '/data/darknet/python/image/'
GENE_IMG_DIR = '../generated/image_expand_obj/'
GENE_XML_DIR = '../generated/xml_expand_obj/'
# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

def rotate_img(img, thealta):
    (h_, w_) = img.shape[:2]
    point_list = [(0, 0), (0, h_), (w_, h_), (w_, 0)]
    center = (w_ // 2, h_ // 2)
    roted_point_list = [((point[0]-center[0])*np.cos(np.pi*thealta/180) 
                         - (point[1]-center[1])*np.sin(np.pi*thealta/180) 
                         + center[0], (point[0]-center[0])*np.sin(np.pi*thealta/180) 
                         + (point[1]-center[1])*np.cos(np.pi*thealta/180) + center[1]) for point in point_list]
    #print(roted_point_list)

    temp = np.zeros((2, 4))
    temp[0] = [roted_point[0] for roted_point in roted_point_list]
    temp[1] = [roted_point[1] for roted_point in roted_point_list]
    (xmin, xmax, ymin, ymax) = (np.min(temp[0]), np.max(temp[0]), np.min(temp[1]), np.max(temp[1]))
    print('xmin: {0}, xmax: {1}, ymin: {2}, ymax: {3}'.format(xmin, xmax, ymin, ymax))

    roted_h, roted_w = ymax - ymin, xmax - xmin
    print('roted_h: {0}, roted_w: {1}'.format(roted_w, roted_h))
    
    top_bottom, left_right = int((roted_h - h_)/2), int((roted_w - w_)/2)
    print('top_bottom:{0}, left_right_:{1}'.format(top_bottom, left_right))
    padding = lambda arg : max(arg, 0)
    #dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
    #padding(left_right), cv2.BORDER_CONSTANT)
    dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
                             padding(left_right), cv2.BORDER_CONSTANT)
    
    (h, w) = dst.shape[:2]   
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, thealta, 1.0)
    rotated_img = cv2.warpAffine(dst, M, (w, h))
    if(top_bottom < 0):
        top_bottom = abs(top_bottom)
        rotated_img = rotated_img[top_bottom:-top_bottom, :, :]
    if(left_right < 0):
        left_right = abs(left_right)
        rotated_img = rotated_img[:, left_right:-left_right, :]
    return rotated_img


seed_xml_names = os.listdir(SEED_XML_DIR)
cls_to_ratate = ['yellow ball', 'Scorecard']
for xml_name in seed_xml_names:
    img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
    xml_name = SEED_XML_DIR + xml_name
    #print(img_name)
    #print(xml_name)
    tree=ET.parse(xml_name)
    img = cv2.imread(img_name)
    #img = Image.fromarray(img[:,:,(2,1,0)])
    objs = get_obj_from_xml(xml_name)
    
    i = 0
    for obj_element in objs:
        bbox = get_bbox_from_xmlobj(obj_element)           
        img_obj = get_obj_from_image_file(img_name, bbox)
        
        cls = get_cls_from_xmlobj(obj_element)
        
        if (cls in cls_to_ratate):
            print(cls)
            for thealta in [15]:
                img_obj = rotate_img(img_obj, thealta)
                new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)       
                tree = new_xml_etree
                img = np.array(new_pil_img)
        else:
            new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)       
            tree = new_xml_etree
            img = np.array(new_pil_img)
        
        '''new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)       
        tree = new_xml_etree
        img = np.array(new_pil_img)'''
    ## save xml and img
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S_")
    new_pil_img = Image.fromarray(img[:,:,(2,1,0)])
    new_pil_img.save(GENE_IMG_DIR + time_str + img_name.split('/')[-1])
    new_xml_etree.write(GENE_XML_DIR + time_str + xml_name.split('/')[-1]) 
    #break

yellow ball
xmin: -4.5971199489645755, xmax: 39.30422673015113, ymin: -3.752514466235148, ymax: 42.97725933762674
roted_h: 43.901346679115704, roted_w: 46.72977380386189
top_bottom:3, left_right_:4
yellow ball
xmin: -4.079481858759536, xmax: 39.0454076850486, ymin: -3.786588639946082, ymax: 40.0454076850486
roted_h: 43.12488954380814, roted_w: 43.831996324994684
top_bottom:3, left_right_:4
yellow ball
xmin: -4.528971601542711, xmax: 42.52897160154271, ymin: -4.236078382729261, ymax: 44.23607838272926
roted_h: 47.057943203085415, roted_w: 48.47215676545852
top_bottom:4, left_right_:4
new_bbox not suitable, retry...
yellow ball
xmin: -4.5971199489645755, xmax: 38.597119948964576, ymin: -3.7184402925242175, ymax: 43.71844029252422
roted_h: 43.19423989792915, roted_w: 47.43688058504844
top_bottom:3, left_right_:4
yellow ball
xmin: -4.372375077572986, xmax: 36.11355603247047, ymin: -3.527769594843562, ymax: 40.49369542113263
roted_h: 40.48593111004345, roted_w: 44.02146501597619
top_bottom:

Scorecard
xmin: -7.13021302165005, xmax: 105.13021302165005, ymin: -11.523611303851837, ymax: 79.52361130385184
roted_h: 112.2604260433001, roted_w: 91.04722260770367
top_bottom:11, left_right_:7
Scorecard
xmin: -10.412937915976016, xmax: 128.41293791597602, ymin: -13.634763322924002, ymax: 109.634763322924
roted_h: 138.82587583195203, roted_w: 123.269526645848
top_bottom:13, left_right_:10
Scorecard
xmin: -7.164287195360984, xmax: 103.16428719536098, ymin: -11.264792258749317, ymax: 79.26479225874931
roted_h: 110.32857439072197, roted_w: 90.52958451749862
top_bottom:11, left_right_:7
Scorecard
xmin: -10.705831134789477, xmax: 127.67175696107854, ymin: -13.341870104110548, ymax: 111.60068914921307
roted_h: 138.37758809586802, roted_w: 124.94255925332362
top_bottom:13, left_right_:10
yellow ball
xmin: -3.8206628136570124, xmax: 37.82066281365701, ymin: -3.8206628136570124, ymax: 37.82066281365701
roted_h: 41.64132562731402, roted_w: 41.64132562731402
top_bottom:3, left_right_:3
Scorecar

Scorecard
xmin: -13.083051059008028, xmax: 112.0489768852971, ymin: -10.739905308500411, ymax: 124.99872435360294
roted_h: 125.13202794430512, roted_w: 135.73862966210334
top_bottom:10, left_right_:13
Scorecard
xmin: -11.823030007206356, xmax: 108.78895583349542, ymin: -10.651457131952547, ymax: 114.91027617705507
roted_h: 120.61198584070178, roted_w: 125.56173330900762
top_bottom:10, left_right_:11
new_bbox not suitable, retry...
yellow ball
xmin: -4.304226730151125, xmax: 40.0454076850486, ymin: -4.045407685048602, ymax: 41.01133351133767
roted_h: 44.34963441519973, roted_w: 45.05674119638627
top_bottom:4, left_right_:4
new_bbox not suitable, retry...
yellow ball
xmin: -3.8547369873679465, xmax: 35.59591794226542, ymin: -3.595917942265423, ymax: 36.561843768554496
roted_h: 39.45065492963337, roted_w: 40.15776171081992
top_bottom:3, left_right_:3
Scorecard
xmin: -6.973616497680325, xmax: 98.71479745257781, ymin: -10.815302515966138, ymax: 75.78122834225522
roted_h: 105.68841395025814,

Scorecard
xmin: -10.324489739428174, xmax: 118.32448973942817, ymin: -12.37474227112233, ymax: 106.37474227112233
roted_h: 128.64897947885635, roted_w: 118.74948454224466
top_bottom:12, left_right_:10
new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
Scorecard
xmin: -10.256341392006306, xmax: 122.96344817319284, ymin: -12.926454535038303, ymax: 106.1511994064299
roted_h: 133.21978956519916, roted_w: 119.0776539414682
top_bottom:13, left_right_:10
Scorecard
xmin: -6.939542323969391, xmax: 101.90546815025846, ymin: -11.040047387357731, ymax: 77.29886643246024
roted_h: 108.84501047422785, roted_w: 88.33891381981798
top_bottom:11, left_right_:6
Scorecard
xmin: -10.44701208968695, xmax: 126.18819304458444, ymin: -13.410018451532409, ymax: 108.37594427782147
roted_h: 136.6352051342714, roted_w: 121.78596272935388
top_bottom:13, left_right_:10
new_bbox not suitable, retry...
Scorecard
xmin: -6.905468150258464, xmax: 103.87139397654752, ymin: -11.2

yellow ball
xmin: -3.8547369873679465, xmax: 35.85473698736794, ymin: -3.5618437685544926, ymax: 37.561843768554496
roted_h: 39.709473974735886, roted_w: 41.12368753710899
top_bottom:3, left_right_:3
yellow ball
xmin: -4.270152556440191, xmax: 42.01133351133767, ymin: -4.304226730151125, ymax: 41.270152556440195
roted_h: 46.28148606777786, roted_w: 45.57437928659132
top_bottom:4, left_right_:4
yellow ball
xmin: -4.338300903862056, xmax: 38.079481858759536, ymin: -3.786588639946082, ymax: 40.75251446623515
roted_h: 42.41778276262159, roted_w: 44.53910310618123
top_bottom:3, left_right_:4
yellow ball
xmin: -4.563045775253645, xmax: 41.52897160154271, ymin: -3.977259337626741, ymax: 44.23607838272926
roted_h: 46.09201737679635, roted_w: 48.213337720356
top_bottom:4, left_right_:4
new_bbox not suitable, retry...
yellow ball
xmin: -4.338300903862056, xmax: 38.338300903862056, ymin: -3.752514466235148, ymax: 41.75251446623515
roted_h: 42.67660180772411, roted_w: 45.505028932470296
top_bottom

...

new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
Scorecard
xmin: -13.430318280658412, xmax: 121.1714992355559, ymin: -11.999926360302084, ymax: 128.96585218659115
roted_h: 134.6018175162143, roted_w: 140.96577854689323
top_bottom:11, left_right_:13
Scorecard
xmin: -13.464392454369346, xmax: 120.43031828065841, ymin: -11.707033141488623, ymax: 129.96585218659115
roted_h: 133.89471073502776, roted_w: 141.67288532807976
top_bottom:11, left_right_:13
new_bbox not suitable, retry...
Scorecard
xmin: -13.205573409266819, xmax: 119.20557340926682, ymin: -11.741107315199557, ymax: 127.74110731519956
roted_h: 132.41114681853364, roted_w: 139.4822146303991
top_bottom:11, left_right_:13
Scorecard
xmin: -12.980828537875226, xmax: 117.94675436416429, ymin: -11.516362443807978, ymax: 125.77518148891049
roted_h: 130.92758290203952, roted_w: 137.29154393271847
top_bottom:11, left_right_:12
Scorecard
xmin: -13.185273580140823, xmax: 106.1511994064299, ymin:

Scorecard
xmin: -13.062751229882032, xmax: 97.06275122988202, ymin: -8.9622461664937, ymax: 120.9622461664937
roted_h: 110.12550245976405, roted_w: 129.9244923329874
top_bottom:8, left_right_:13
new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
Scorecard
xmin: -13.151199406429889, xmax: 107.1511994064299, ymin: -10.222267218295372, ymax: 124.22226721829537
roted_h: 120.30239881285979, roted_w: 134.44453443659074
top_bottom:10, left_right_:13
new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
Scorecard
xmin: -13.859508194315595, xmax: 111.60068914921307, ymin: -10.671756961078543, ymax: 129.63768278736762
roted_h: 125.46019734352866, roted_w: 140.30943974844615
top_bottom:10, left_right_:13
Scorecard
xmin: -13.396244106947478, xmax: 124.36216993323654, ymin: -12.224671231693662, ymax: 130.4834902767962
roted_h: 137.75841404018402, roted_w: 142.70816150848987
top_bot

new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
new_bbox not suitable, retry...
yellow ball
xmin: -5.182906386591483, xmax: 35.89001316777803, ymin: -3.1667280286082473, ymax: 46.39147289999983
roted_h: 41.07291955436951, roted_w: 49.55820092860808
top_bottom:3, left_right_:5
yellow ball
xmin: -4.079481858759536, xmax: 38.78658863994608, ymin: -3.8206628136570124, ymax: 39.0454076850486
roted_h: 42.86607049870562, roted_w: 42.86607049870561
top_bottom:3, left_right_:3
yellow ball
xmin: -4.40644925128392, xmax: 35.37237507757298, ymin: -3.2348763760301082, ymax: 41.49369542113263
roted_h: 39.7788243288569, roted_w: 44.728571797162736
top_bottom:3, left_right_:4
yellow ball
xmin: -4.079481858759536, xmax: 38.78658863994608, ymin: -3.8206628136570124, ymax: 39.0454076850486
roted_h: 42.86607049870562, roted_w: 42.86607049870561
top_bottom:3, left_right_:3
yellow ball
xmin: -4.147630206181397, xmax: 34.85473698

Scorecard
xmin: -10.412937915976016, xmax: 128.41293791597602, ymin: -13.634763322924002, ymax: 109.634763322924
roted_h: 138.82587583195203, roted_w: 123.269526645848
top_bottom:13, left_right_:10
Scorecard
xmin: -7.164287195360984, xmax: 103.87139397654752, ymin: -11.298866432460251, ymax: 78.52361130385184
roted_h: 111.03568117190851, roted_w: 89.8224777363121
top_bottom:11, left_right_:7
Scorecard
xmin: -7.164287195360984, xmax: 103.87139397654752, ymin: -11.298866432460251, ymax: 78.52361130385184
roted_h: 111.03568117190851, roted_w: 89.8224777363121
top_bottom:11, left_right_:7
new_bbox not suitable, retry...
Scorecard
xmin: -10.44701208968695, xmax: 127.41293791597602, ymin: -13.375944277821482, ymax: 109.634763322924
roted_h: 137.85995000566297, roted_w: 123.01070760074549
top_bottom:13, left_right_:10
yellow ball
xmin: -4.079481858759536, xmax: 39.0454076850486, ymin: -3.786588639946082, ymax: 40.0454076850486
roted_h: 43.12488954380814, roted_w: 43.831996324994684
top_bottom

Scorecard
xmin: -11.053098356439861, xmax: 136.79427931133733, ymin: -14.601891155912227, ymax: 115.5678169822013
roted_h: 147.8473776677772, roted_w: 130.16970813811352
top_bottom:14, left_right_:10
Scorecard
xmin: -7.838521809535749, xmax: 110.80444763582481, ymin: -11.939026872924089, ymax: 86.1978459180266
roted_h: 118.64296944536056, roted_w: 98.13687279095069
top_bottom:12, left_right_:7
new_bbox not suitable, retry...
